In [1]:
import os
import pandas as pd
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from tqdm import tqdm
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import clipboard
import base64
from io import BytesIO
from PIL import Image
from proxyattention.meta_utils import read_pickle, return_grouped_results


In [12]:
!python result_aggregator.py

100%|███████████████████████████████████████████| 19/19 [00:10<00:00,  1.88it/s]


In [2]:
combined_df = read_pickle(fname = "./results/aggregated_runs.csv")[0]


In [3]:
combined_df.shape

(561, 39)

# Clean if required

In [4]:
broken_runs = list(combined_df[combined_df["global_run_count"].isna() == True]["index"].values)
broken_runs[:2]

['/media/eragon/data/CODE/thesis_runs/proper_runs/hypers_01052023_192308/events.out.tfevents.1682962474.pop-os',
 '/media/eragon/data/CODE/thesis_runs/proper_runs/proxy_run_28042023_165052/events.out.tfevents.1682693458.pop-os']

In [5]:
combined_df = combined_df[combined_df["global_run_count"].isna() == False]
combined_df["global_run_count"] = combined_df["global_run_count"].astype(int)
combined_df["global_run_count"].unique()

array([ 5,  6, 15, 16, 25, 26, 35, 36, 40, 20, 21,  1,  8])

In [6]:
broken_runs.extend(combined_df.query('global_run_count < 10')["index"].values)
len(broken_runs)

119

In [7]:
# Delete broken runs
[Path.unlink(Path(x)) for x in broken_runs]

FileNotFoundError: [Errno 2] No such file or directory: '/media/eragon/data/CODE/thesis_runs/proper_runs/hypers_01052023_192308/events.out.tfevents.1682962474.pop-os'

In [103]:
def dir_empty(path):
    empty = True
    for item in path.glob('*'):
        if item.is_file():
            empty = False
        if item.is_dir() and not dir_empty(item):
            empty = False
    # if empty:
        # path.rmdir()  # Remove if you just want to have the result
    return empty

# dir_empty([x for x in Path("./results").glob("*") if x.is_dir()])
# [Path.rmdir(x) for x in Path("./results").glob("*") if x.is_dir() and dir_empty(x)]
[Path.rmdir(x) for x in Path("./runs").glob("*") if x.is_dir() and dir_empty(x)]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

# Continue here

In [8]:
combined_df.head()

,index,proxy_step,Loss/Train,Acc/Train,global_run_count,Loss/Val,Acc/Val,experiment_name,image_size,batch_size,...,name_fn,writer,label_map,rev_label_map,num_classes,dataset_sizes,criterion,save_path,final_acc,Number_Chosen
0,/media/eragon/data/CODE/thesis_runs/proper_run...,False,0.004473,94.371521,5,0.008572,83.978638,hypers,224,64,...,<function get_parent_name at 0x7fd559fca680>,<tensorboardX.writer.SummaryWriter object at 0...,"{0: 'n02085620-Chihuahua', 1: 'n02085936-Malte...","{'n02085620-Chihuahua': 0, 'n02085936-Maltese_...",51,"{'train': 4495, 'val': 4494}",CrossEntropyLoss(),/media/eragon/data/CODE/thesis_runs/proper_run...,83.97863818424567,NaN
1,/media/eragon/data/CODE/thesis_runs/proper_run...,True,0.006403,89.454948,6,0.004788,91.522026,hypers,224,64,...,<function get_parent_name at 0x7fd559fca680>,<tensorboardX.writer.SummaryWriter object at 0...,"{0: 'n02085620-Chihuahua', 1: 'n02085936-Malte...","{'n02085620-Chihuahua': 0, 'n02085936-Maltese_...",51,"{'train': 4495, 'val': 4494}",CrossEntropyLoss(),/media/eragon/data/CODE/thesis_runs/proper_run...,91.52202937249666,380.0
2,/media/eragon/data/CODE/thesis_runs/proper_run...,False,0.000564,99.666298,15,0.00361,93.257675,hypers,224,64,...,<function get_parent_name at 0x7fd559fca680>,<tensorboardX.writer.SummaryWriter object at 0...,"{0: 'n02085620-Chihuahua', 1: 'n02085936-Malte...","{'n02085620-Chihuahua': 0, 'n02085936-Maltese_...",51,"{'train': 4495, 'val': 4494}",CrossEntropyLoss(),/media/eragon/data/CODE/thesis_runs/proper_run...,93.25767690253672,NaN
3,/media/eragon/data/CODE/thesis_runs/proper_run...,True,0.004289,92.593384,16,0.003292,93.979507,hypers,224,64,...,<function get_parent_name at 0x7fd559fca680>,<tensorboardX.writer.SummaryWriter object at 0...,"{0: 'n02085620-Chihuahua', 1: 'n02085936-Malte...","{'n02085620-Chihuahua': 0, 'n02085936-Maltese_...",51,"{'train': 4685, 'val': 4684}",CrossEntropyLoss(),/media/eragon/data/CODE/thesis_runs/proper_run...,93.97950469684031,278.0
4,/media/eragon/data/CODE/thesis_runs/proper_run...,False,0.000234,99.866516,25,0.001474,97.374275,hypers,224,64,...,<function get_parent_name at 0x7fd559fca680>,<tensorboardX.writer.SummaryWriter object at 0...,"{0: 'n02085620-Chihuahua', 1: 'n02085936-Malte...","{'n02085620-Chihuahua': 0, 'n02085936-Maltese_...",51,"{'train': 4495, 'val': 4494}",CrossEntropyLoss(),/media/eragon/data/CODE/thesis_runs/proper_run...,97.37427681352915,NaN


In [9]:
combined_df.shape

(552, 39)

In [10]:
combined_df[combined_df["global_run_count"]!= "0"]["proxy_steps"].value_counts()

[5, 'p', 9, 'p', 9, 'p', 9, 'p', 4]    491
[20, 'p', 19]                           44
[40]                                    15
Name: proxy_steps, dtype: int64

# Image code

In [21]:
# filtered_df.iloc[1].original_images

In [22]:
# filtered_df.iloc[1].converted_proxy

# Preprocess

In [11]:
def check_proxy(string): return "p" in str(string)

In [12]:
def calc_stats(values):
    return f"min: {values.min()} \nmax: {values.max()} \navg: {values.mean()}"

In [13]:
def convert_float(df, cols, totype= float):
    for col in cols:
        df[col] = df[col].astype(totype)

In [14]:
combined_df = combined_df.fillna(0)
# col to check for proxy
# combined_df["has_proxy"] = combined_df["proxy_steps"].apply(check_proxy)
# Fix naming
combined_df = combined_df.rename(columns={"Acc/Val":"accuracy", "proxy_steps":"step_schedule"})
# Fix types
convert_float(combined_df, ["change_subset_attention", "proxy_threshold", "accuracy"], float)
convert_float(combined_df, ["global_run_count"], int)
convert_float(combined_df, ["transfer_imagenet"], bool)

# ignore failed runs
# combined_df = combined_df[combined_df["global_run_count"]!=0]


In [15]:
combined_df["has_proxy"] = combined_df["step_schedule"].apply(check_proxy)

In [16]:
combined_df.loc[combined_df["has_proxy"] == False, ["pixel_replacement_method", "gradient_method"]] = "baseline"

# Grouped Results

In [17]:
# def return_grouped_results(df, group_cols ,filter = None, index_cols = (["ds_name", ("accuracy")]), print_latex = False):
#     if filter != None:
#         df = df.reset_index()
#         for key in filter.keys():
#             df = df[df[key] == filter[key]]
#     final_df = pd.DataFrame(df.groupby(group_cols, as_index=True).mean(numeric_only = True)["accuracy"]).sort_values(index_cols, ascending=False)
#     if print_latex == True:
#         clipboard.copy(final_df.to_latex())

#     return final_df



In [18]:
return_grouped_results(combined_df, ["ds_name","model", "gradient_method", "step_schedule", "change_subset_attention"])

accuracy
ds_name      model           gradient_method step_schedule                       change_subset_attention           
plantdisease resnet18        gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                     99.147778
             efficientnet_b0 gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                     99.101112
             resnet50        gradcamplusplus [20, 'p', 19]                       0.80                     99.023333
             resnet18        baseline        [40]                                0.80                     98.949997
             resnet50        baseline        [40]                                0.80                     98.949997
             efficientnet_b0 baseline        [40]                                0.80                     98.900002
             resnet18        gradcamplusplus [20, 'p', 19]                       0.80                     98.826668
             resnet50        gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                     98.786666
             efficientnet_b0 gradcamplusplus [20, 'p', 19]                       0.80                     98.680000
dogs         resnet50        gradcamplusplus [20, 'p', 19]                       0.80                     95.497701
                                             [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                     95.495351
             efficientnet_b0 gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.20                     94.672288
                                                                                 0.80                     93.442382
                                                                                 0.95                     93.339572
             resnet50        baseline        [40]                                0.80                     92.011574
             efficientnet_b0 gradcamplusplus [20, 'p', 19]                       0.80                     91.410772
             resnet18        gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                     87.411429
                             gradcam         [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                     86.384286
             efficientnet_b0 baseline        [40]                                0.80                     85.825546
             resnet18        gradcam         [20, 'p', 19]                       0.80                     80.473335
                             baseline        [40]                                0.80                     71.414997
cifar100     resnet50        gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                     83.106666
                                             [20, 'p', 19]                       0.80                     82.090001
             efficientnet_b0 gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                     81.665000
                                             [20, 'p', 19]                       0.80                     79.037500
             resnet50        baseline        [40]                                0.80                     78.699997
             resnet18        gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                     76.778000
                                             [20, 'p', 19]                       0.80                     75.559998
             efficientnet_b0 baseline        [40]                                0.80                     75.220001
             resnet18        baseline        [40]                                0.80                     71.000000
caltech101   efficientnet_b0 gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                     97.773901
             resnet50        gradcamplusplus [20, 'p', 19]                       0.80                     97.543017
                                             [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                     96.790467
             efficientnet_b0 gradcamplusplus [2

In [19]:
return_grouped_results(combined_df, ["ds_name", "pixel_replacement_method", "transfer_imagenet", "gradient_method", "step_schedule", "change_subset_attention", "proxy_threshold", "model", "proxy_image_weight"], filter = { "ds_name":"cifar100"})

accuracy
ds_name  pixel_replacement_method transfer_imagenet gradient_method step_schedule                       change_subset_attention proxy_threshold model           proxy_image_weight           
cifar100 blended                  True              gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.8                     0.85            resnet50        0.2                 83.106666
                                                                    [20, 'p', 19]                       0.8                     0.85            resnet50        0.2                 82.090001
                                                                    [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.8                     0.85            efficientnet_b0 0.2                 81.665000
                                                                    [20, 'p', 19]                       0.8                     0.85            efficientnet_b0 0.2                 79.037500
         baseline                 True              baseline        [40]                                0.8                     0.85            resnet50        0.2                 78.699997
         blended                  True              gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.8                     0.85            resnet18        0.2                 76.778000
                                                                    [20, 'p', 19]                       0.8                     0.85            resnet18        0.2                 75.559998
         baseline                 True              baseline        [40]                                0.8                     0.85            efficientnet_b0 0.2                 75.220001
                                                                                                                                                resnet18        0.2                 71.000000

In [21]:
return_grouped_results(combined_df, ["ds_name", "pixel_replacement_method", "transfer_imagenet", "gradient_method", "step_schedule", "change_subset_attention", "proxy_threshold", "model", "proxy_image_weight"], filter = {"model":"efficientnet_b0"})

accuracy
ds_name      pixel_replacement_method transfer_imagenet gradient_method step_schedule                       change_subset_attention proxy_threshold model           proxy_image_weight           
plantdisease blended                  True              gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.80                    0.85            efficientnet_b0 0.2                 99.101112
             baseline                 True              baseline        [40]                                0.80                    0.85            efficientnet_b0 0.2                 98.900002
             blended                  True              gradcamplusplus [20, 'p', 19]                       0.80                    0.85            efficientnet_b0 0.2                 98.680000
dogs         blended                  True              gradcamplusplus [5, 'p', 9, 'p', 9, 'p', 9, 'p', 4] 0.20                    0.80            efficientnet_b0 0.8                 94.985541
                                                                                                                                    0.10            efficientnet_b0 0.1                 94.911736
                                                                                                                                    0.40            efficientnet_b0 0.95                94.890021
                                                                                                                                                                    0.1                 94.845623
                                                                                                                                    0.10            efficientnet_b0 0.8                 94.786770
                                                                                                                                    0.40            efficientnet_b0 0.8                 94.782388
                                                                                                                                    0.85            efficientnet_b0 0.4                 94.762321
                                                                                                                                    0.10            efficientnet_b0 0.4                 94.737558
                                                                                                                                    0.80            efficientnet_b0 0.1                 94.723004
                                                                                                                                    0.85            efficientnet_b0 0.1                 94.658141
                                                                                                                                                                    0.8                 94.612412
                                                                                                                                    0.10            efficientnet_b0 0.95                94.565847
                                                                                                                                    0.80            efficientnet_b0 0.4                 94.493133
                                                                                                                                    0.40            efficientnet_b0 0.4                 94.491127
                                                                                                                                    0.80            efficientnet_b0 0.95                94.342921
                                                                                                            0.80                    0.10            efficientnet_b0 0.1                 94.259889
                                                                                                            0.2

In [27]:
pd.set_option('display.max_rows', 500)
return_grouped_results(combined_df, ["ds_name", "pixel_replacement_method", "transfer_imagenet", "gradient_method", "step_schedule", "change_subset_attention", "proxy_threshold", "model", "proxy_image_weight"])

accuracy
ds_name      pixel_replacement_method transfer_imagenet gradient_method step_schedule change_subset_attention proxy_threshold model                proxy_image_weight            
plantdisease baseline                 baseline          baseline        [20]          0.8                     0.85            efficientnet_b0      0.1                  99.349998
                                                                                                                              resnet50             0.1                  99.220001
                                                                                                                              resnet18             0.1                  98.930000
             blended                  True              gradcamplusplus [10, 'p', 10] 0.8                     0.10            resnet18             0.2                  98.866666
                                                                                                              0.85            resnet18             0.95                 98.836665
                                                                                                                                                   0.2                  98.813334
                                                                                                              0.10            efficientnet_b0      0.2                  98.770002
                                                                                                                              resnet18             0.95                 98.633334
imagenette   blended                  True              gradcamplusplus [10, 'p', 10] 0.8                     0.10            efficientnet_b0      0.2                  98.831151
                                                                                                              0.85            resnet18             0.95                 97.782003
                                                                                                              0.10            resnet18             0.95                 97.641177
                                                                                                              0.85            resnet18             0.2                  97.528516
                                                                                                              0.10            resnet18             0.2                  97.042669
             baseline                 baseline          baseline        [20]          0.8                     0.85            resnet50             0.1                  94.190956
                                                                                                                              resnet18             0.1                  93.409378
                                                                                                                              efficientnet_b0      0.1                  88.001686
                                                                                                                              vit_base_patch16_224 0.1                  39.543728
                                                                                                                              vgg16                0.1                  10.318969
dogs         blended                  True              gradcamplusplus [10, 'p', 10] 0.2                     0.85            efficientnet_b0      0.95                 85.238714
                                                                                      0.8                     0.85            efficientnet_b0      0.95                 85.088000
                                                                                                                                                   0.2                  85.081248
                                                                           

In [122]:

return_grouped_results(combined_df, ["ds_name", "pixel_replacement_method", "transfer_imagenet", "gradient_method", "step_schedule", "change_subset_attention", "proxy_threshold"], filter = { "model":"efficientnet_b0", "has_proxy":True,})

accuracy
ds_name      pixel_replacement_method transfer_imagenet gradient_method step_schedule change_subset_attention proxy_threshold           
plantdisease blended                  True              gradcamplusplus [10, 'p', 10] 0.8                     0.10             98.770002
imagenette   blended                  True              gradcamplusplus [10, 'p', 10] 0.8                     0.10             98.831151
dogs         blended                  True              gradcamplusplus [10, 'p', 10] 0.2                     0.85             85.131791
                                                                                      0.8                     0.85             85.083949
                                                                                      0.2                     0.10             85.020728
                                                                                      0.8                     0.10             75.658566
                                                                        [20, 'p', 19] 0.8                     0.10             11.996667
cifar100     blended                  True              gradcamplusplus [10, 'p', 10] 0.2                     0.10             80.146666
                                                                                                              0.85             80.126668
                                                                                      0.8                     0.10             80.078334
                                                                                                              0.85             80.014999
caltech101   blended                  True              gradcamplusplus [10, 'p', 10] 0.8                     0.10             96.289004
                                                                                                              0.85             95.751310
asl          blended                  True              gradcamplusplus [10, 'p', 10] 0.8                     0.10             99.980001

In [123]:
return_grouped_results(combined_df, ["ds_name", "pixel_replacement_method", "transfer_imagenet", "gradient_method", "step_schedule", "model"])

accuracy
ds_name      pixel_replacement_method transfer_imagenet gradient_method step_schedule model                           
plantdisease baseline                 baseline          baseline        [20]          efficientnet_b0        99.349998
                                                                                      resnet50               99.220001
                                                                                      resnet18               98.930000
             blended                  True              gradcamplusplus [10, 'p', 10] resnet18               98.787500
                                                                                      efficientnet_b0        98.770002
imagenette   blended                  True              gradcamplusplus [10, 'p', 10] efficientnet_b0        98.831151
                                                                                      resnet18               97.498591
             baseline                 baseline          baseline        [20]          resnet50               94.190956
                                                                                      resnet18               93.409378
                                                                                      efficientnet_b0        88.001686
                                                                                      vit_base_patch16_224   39.543728
                                                                                      vgg16                  10.318969
dogs         blended                  True              gradcamplusplus [10, 'p', 10] efficientnet_b0        82.104274
                                                                                      resnet18               79.480000
             baseline                 baseline          baseline        [20]          resnet50               66.629997
                                                                                      resnet18               63.660000
                                                                                      efficientnet_b0        31.123332
                                                                                      vgg16                  15.650000
                                                                        [40]          efficientnet_b0        14.920000
             blended                  True              gradcamplusplus [20, 'p', 19] efficientnet_b0        11.996667
             baseline                 baseline          baseline        [20]          vit_base_patch16_224   10.950000
cifar100     blended                  True              gradcamplusplus [10, 'p', 10] resnet50               80.795834
                                                                                      efficientnet_b0        80.091667
                                                                                      resnet18               75.286540
             baseline                 baseline          baseline        [20]          efficientnet_b0        73.320000
                                                                                      resnet18               65.949997
                                                                                      resnet50               63.660000
                                                                                      vit_base_patch16_224   13.120000
                                                                                      vgg16                   1.040000
caltech101   blended                  True              gradcamplusplus [10, 'p', 10] efficientnet_b0        95.981750
                                                                                      resnet18               95.683872
             baseline                 baseline          baseline        [20]          efficientnet_b0        92.454071
                                                                

In [33]:
return_grouped_results(combined_df, ["ds_name", "pixel_replacement_method", "transfer_imagenet", "gradient_method", "change_subset_attention", "step_schedule", "proxy_image_weight",  "model", "global_run_count"])

accuracy
ds_name      pixel_replacement_method transfer_imagenet gradient_method change_subset_attention step_schedule proxy_image_weight model                global_run_count            
plantdisease baseline                 baseline          baseline        0.8                     [20]          0.1                efficientnet_b0      20                 99.349998
                                                                                                                                 resnet50             20                 99.220001
                                                                                                                                 resnet18             20                 98.930000
imagenette   baseline                 baseline          baseline        0.8                     [20]          0.1                resnet50             20                 94.190956
                                                                                                                                 resnet18             20                 93.409378
                                                                                                                                 efficientnet_b0      20                 88.001686
                                                                                                                                 vit_base_patch16_224 20                 39.543728
                                                                                                                                 vgg16                20                 10.318969
dogs         baseline                 baseline          baseline        0.8                     [20]          0.1                efficientnet_b0      20                 71.519997
                                                                                                                                 resnet50             20                 66.629997
                                                                                                                                 resnet18             20                 63.660000
                                                                                                                                 vgg16                20                 15.650000
                                                                                                                                 vit_base_patch16_224 20                 10.950000
cifar100     baseline                 baseline          baseline        0.8                     [20]          0.1                efficientnet_b0      20                 73.320000
                                                                                                                                 resnet18             20                 65.949997
                                                                                                                                 resnet50             20                 63.660000
                                                                                                                                 vit_base_patch16_224 20                 13.120000
                                                                                                                                 vgg16                20                  1.040000
caltech101   baseline                 baseline          baseline        0.8                     [20]          0.1                efficientnet_b0      20                 92.454071
                                                                                                                                 resnet18             20                 88.954506
                                                                                                                                 resnet50             20                 85.454941
                                                     

# Compare CAM

In [34]:
test_models = return_grouped_results(combined_df, ["index","ds_name", "pixel_replacement_method", "transfer_imagenet", "gradient_method", "step_schedule", "model", "num_classes"])

test_models = test_models.reset_index()

In [35]:
test_models

,index,ds_name,pixel_replacement_method,transfer_imagenet,gradient_method,step_schedule,model,num_classes,accuracy
0,runs/baseline_run_27032023_143750/events.out.t...,plantdisease,baseline,baseline,baseline,[20],efficientnet_b0,39,99.349998
1,runs/baseline_run_27032023_144800/events.out.t...,plantdisease,baseline,baseline,baseline,[20],resnet50,39,99.220001
2,runs/baseline_run_27032023_142831/events.out.t...,plantdisease,baseline,baseline,baseline,[20],resnet18,39,98.930000
3,runs/baseline_run_26032023_215855/events.out.t...,imagenette,baseline,baseline,baseline,[20],resnet50,10,94.190956
4,runs/baseline_run_26032023_135310/events.out.t...,imagenette,baseline,baseline,baseline,[20],resnet18,10,93.409378
5,runs/baseline_run_26032023_205454/events.out.t...,imagenette,baseline,baseline,baseline,[20],efficientnet_b0,10,88.001686
6,runs/baseline_run_27032023_114723/events.out.t...,imagenette,baseline,baseline,baseline,[20],vit_base_patch16_224,10,39.543728
7,runs/baseline_run_27032023_133419/events.out.t...,imagenette,baseline,baseline,baseline,[20],vgg16,10,10.477398
8,runs/baseline_run_27032023_131945/events.out.t...,imagenette,baseline,baseline,baseline,[20],vgg16,10,10.160541
9,runs/baseline_run_26032023_193957/events.out.t...,dogs,baseline,baseline,baseline,[20],efficientnet_b0,120,71.519997


: 

In [35]:
model_1, model_2 = test_models.loc[:1].values

In [36]:
model_1

array(['runs/baseline_run_27032023_143750/events.out.tfevents.1679920670.eragon',
       'plantdisease', 'baseline', 'baseline', 'baseline', '[20]',
       'efficientnet_b0', '39', 99.3499984741211], dtype=object)

In [1]:
import torch
from proxyattention.training import choose_network

import albumentations as A

from albumentations.pytorch import ToTensorV2
from proxyattention.data_utils import ImageClassDs, create_folds, get_parent_name

/home/eragon/mambaforge/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
arch_1 = choose_network({"model":model_2[-3] , "transfer_imagenet":False, "num_classes": int(model_2[-2]), "device":"cuda:0"})
model_weights = torch.load(Path(model_2[0]).parent/"checkpoint", map_location='cuda:0')
arch_1.load_state_dict(model_weights['model_state_dict'])
arch_1.eval();

RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "conv1.weight", "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "layer1.0.conv1.weight", "layer1.0.bn1.weight", "layer1.0.bn1.bias", "layer1.0.bn1.running_mean", "layer1.0.bn1.running_var", "layer1.0.conv2.weight", "layer1.0.bn2.weight", "layer1.0.bn2.bias", "layer1.0.bn2.running_mean", "layer1.0.bn2.running_var", "layer1.0.conv3.weight", "layer1.0.bn3.weight", "layer1.0.bn3.bias", "layer1.0.bn3.running_mean", "layer1.0.bn3.running_var", "layer1.0.downsample.0.weight", "layer1.0.downsample.1.weight", "layer1.0.downsample.1.bias", "layer1.0.downsample.1.running_mean", "layer1.0.downsample.1.running_var", "layer1.1.conv1.weight", "layer1.1.bn1.weight", "layer1.1.bn1.bias", "layer1.1.bn1.running_mean", "layer1.1.bn1.running_var", "layer1.1.conv2.weight", "layer1.1.bn2.weight", "layer1.1.bn2.bias", "layer1.1.bn2.running_mean", "layer1.1.bn2.running_var", "layer1.1.conv3.weight", "layer1

In [126]:
config = {"image_size": 224, "subset_images":100, "load_proxy_data":False, "name_fn":get_parent_name, "ds_path": "/Users/eragon/Documents/CODE/Datasets/imagenette2-320/val", "batch_size":10}

In [118]:
data_transforms = A.Compose(
            [
                A.Resize(config["image_size"], config["image_size"]),
                A.CenterCrop(config["image_size"], config["image_size"], p=1.0),
                A.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225],
                    max_pixel_value=255.0,
                    p=1.0,
                ),
                ToTensorV2(p=1.0),
            ],
            p=1.0,
        ),


In [119]:
val = create_folds(config=config)

In [123]:
image_datasets = ImageClassDs(
            val[0], config["ds_path"], train=False, transforms=data_transforms
        ),


In [127]:
val_dls = torch.utils.data.DataLoader(
        image_datasets,
        batch_size=config["batch_size"],
        shuffle=False,
        num_workers=8,
    ),


In [1]:
arch_1(val_dls)

NameError: name 'arch_1' is not defined